<a href="https://colab.research.google.com/github/XGHouFTW/py-cryptopredict/blob/main/Merging_Dates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import datetime
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
basepath = "/content/drive/MyDrive/ISEF/Datasets/Stonks/Preprocessed/CryptoStockPrice.csv"
dfbase = pd.read_csv(basepath)
#dfbase = dfbase.iloc[0:548,:] #Training set from Jan 31 2020 to July 31 2021

medium = {"news":"/content/drive/MyDrive/ISEF/Datasets/Google News/","reddit":"/content/drive/MyDrive/ISEF/Datasets/Reddit/Reddit V3/Reformatted Dates/"}
keyword = ["bitcoin","ethereum","dogecoin","cryptocurrency","economy","finance","politics","pandemic"]
scorelist = {} #dictionary of scorecolumnname and path
for med in medium:
  for word in keyword:
    scorelist[med + "-" + word] = medium[med] + med + "-" + word + "-scored.csv"
scorelist

{'news-bitcoin': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-bitcoin-scored.csv',
 'news-cryptocurrency': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-cryptocurrency-scored.csv',
 'news-dogecoin': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-dogecoin-scored.csv',
 'news-economy': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-economy-scored.csv',
 'news-ethereum': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-ethereum-scored.csv',
 'news-finance': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-finance-scored.csv',
 'news-pandemic': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-pandemic-scored.csv',
 'news-politics': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-politics-scored.csv',
 'reddit-bitcoin': '/content/drive/MyDrive/ISEF/Datasets/Reddit/Reddit V3/Reformatted Dates/reddit-bitcoin-scored.csv',
 'reddit-cryptocurrency': '/content/drive/MyDrive/ISEF/Datasets/Reddit/Reddit V3/Reformatted Dates/reddit-cryptocu

In [3]:
scorelist = {'news-bitcoin': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-bitcoin-scored.csv',
 'news-cryptocurrency': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-cryptocurrency-scored.csv',
 'news-dogecoin': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-dogecoin-scored.csv',
 'news-economy': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-economy-scored.csv',
 'news-ethereum': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-ethereum-scored.csv',
 'news-finance': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-finance-scored.csv',
 'news-pandemic': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-pandemic-scored.csv',
 'news-politics': '/content/drive/MyDrive/ISEF/Datasets/Google News/news-politics-scored.csv'}

scorelist2 = {'reddit-bitcoin': '/content/drive/MyDrive/ISEF/Datasets/Reddit/Reddit V3/Reformatted Dates/reddit-bitcoin-scored.csv',
 'reddit-cryptocurrency': '/content/drive/MyDrive/ISEF/Datasets/Reddit/Reddit V3/Reformatted Dates/reddit-cryptocurrency-scored.csv',
 'reddit-dogecoin': '/content/drive/MyDrive/ISEF/Datasets/Reddit/Reddit V3/Reformatted Dates/reddit-dogecoin-scored.csv',
 'reddit-economy': '/content/drive/MyDrive/ISEF/Datasets/Reddit/Reddit V3/Reformatted Dates/reddit-economy-scored.csv',
 'reddit-ethereum': '/content/drive/MyDrive/ISEF/Datasets/Reddit/Reddit V3/Reformatted Dates/reddit-ethereum-scored.csv',
 'reddit-finance': '/content/drive/MyDrive/ISEF/Datasets/Reddit/Reddit V3/Reformatted Dates/reddit-finance-scored.csv',
 'reddit-pandemic': '/content/drive/MyDrive/ISEF/Datasets/Reddit/Reddit V3/Reformatted Dates/reddit-pandemic-scored.csv',
 'reddit-politics': '/content/drive/MyDrive/ISEF/Datasets/Reddit/Reddit V3/Reformatted Dates/reddit-politics-scored.csv'}

In [4]:
transMonth = {"Jan":"01","Feb":"02","Mar":"03","Apr":"04","May":"05","Jun":"06","Jul":"07","Aug":"08","Sep":"09","Oct":"10","Nov":"11","Dec":"12"}

a = "2020-01-31	" #from historical prices
def trans(strdate):
    return datetime.datetime(int(strdate[0:4]), int(strdate[5:7]), int(strdate[8:10]))

In [5]:
start = trans("2020-01-31") #to be compared to new dates for indexing
for i in range(len(dfbase)):
  dfbase.loc[i,"Date"] = trans(dfbase.loc[i,"Date"])

In [6]:
for scoreset in scorelist:
  print(scoreset)
  print()
  dfAdd = pd.read_csv(scorelist[scoreset])
  dfbase[scoreset + "PosScoresum"] = 0
  dfbase[scoreset + "PosScoreaverage"] = 0
  dfbase[scoreset + "NegScoresum"] = 0
  dfbase[scoreset + "NegScoreaverage"] = 0
  dfbase[scoreset + "ComScoresum"] = 0
  dfbase[scoreset + "ComScoreaverage"] = 0

  dfbase[scoreset + "Scorecount"] = 0

  for i in range(len(dfAdd)):
    ind = (trans(dfAdd.loc[i,"Date"])-start).days
    if ind < 0 or ind > 731:
      print("Date out of range error:" + scoreset + str(i) + dfAdd.loc[i,"Date"])
    else:

      dfbase.loc[ind, scoreset + "PosScoresum"] = dfbase.loc[ind, scoreset + "PosScoresum"] + dfAdd.loc[i,"Pos"] #depends on if we only use compound or others too
      dfbase.loc[ind, scoreset + "NegScoresum"] = dfbase.loc[ind, scoreset + "NegScoresum"] + dfAdd.loc[i,"Neg"] #depends on if we only use compound or others too
      dfbase.loc[ind, scoreset + "ComScoresum"] = dfbase.loc[ind, scoreset + "ComScoresum"] + dfAdd.loc[i,"Com"] #depends on if we only use compound or others too
      dfbase.loc[ind, scoreset + "Scorecount"] = dfbase.loc[ind, scoreset + "Scorecount"] + 1
  
  for i in range(len(dfbase)):
    if not dfbase.loc[i,scoreset + "Scorecount"] == 0:
      dfbase[scoreset + "PosScoreaverage"] = dfbase[scoreset + "PosScoresum"]/dfbase[scoreset + "Scorecount"]
      dfbase[scoreset + "NegScoreaverage"] = dfbase[scoreset + "NegScoresum"]/dfbase[scoreset + "Scorecount"]
      dfbase[scoreset + "ComScoreaverage"] = dfbase[scoreset + "ComScoresum"]/dfbase[scoreset + "Scorecount"]


news-bitcoin

Date out of range error:news-bitcoin141342022-02-10
news-cryptocurrency

Date out of range error:news-cryptocurrency102020-01-29
news-dogecoin

news-economy

Date out of range error:news-economy153992022-02-21
news-ethereum

Date out of range error:news-ethereum13192022-02-05
Date out of range error:news-ethereum81772022-02-09
Date out of range error:news-ethereum93392022-02-10
news-finance

Date out of range error:news-finance142452022-02-09
Date out of range error:news-finance160112022-02-10
Date out of range error:news-finance165642022-02-15
Date out of range error:news-finance176222022-02-17
news-pandemic

Date out of range error:news-pandemic58872022-02-15
Date out of range error:news-pandemic116072022-02-02
Date out of range error:news-pandemic167472022-02-21
Date out of range error:news-pandemic169652022-02-07
news-politics

Date out of range error:news-politics4682022-02-16
Date out of range error:news-politics74052022-02-10
Date out of range error:news-politics10

In [7]:
dfbase

,Unnamed: 0,Date,BTC-Open,BTC-High,BTC-Low,BTC-Close,BTC-Adj Close,BTC-Volume,ETH-Open,ETH-High,...,news-pandemicComScoresum,news-pandemicComScoreaverage,news-pandemicScorecount,news-politicsPosScoresum,news-politicsPosScoreaverage,news-politicsNegScoresum,news-politicsNegScoreaverage,news-politicsComScoresum,news-politicsComScoreaverage,news-politicsScorecount
0,0,2020-01-31 00:00:00,9508.313477,9521.706055,9230.776367,9350.529297,9350.529297,29432489719,184.736908,185.405838,...,-1.3965,-0.099750,14,2.012,0.069379,1.795,0.061897,0.3532,0.012179,29
1,1,2020-02-01 00:00:00,9346.357422,9439.323242,9313.239258,9392.875000,9392.875000,25922656496,180.113770,183.845551,...,0.0000,0.000000,2,0.930,0.066429,0.894,0.063857,0.3759,0.026850,14
2,2,2020-02-02 00:00:00,9389.820313,9468.797852,9217.824219,9344.365234,9344.365234,30835736946,183.532501,193.080399,...,0.0880,0.022000,4,1.033,0.068867,1.305,0.087000,-0.9999,-0.066660,15
3,3,2020-02-03 00:00:00,9344.683594,9540.372070,9248.633789,9293.521484,9293.521484,30934096509,188.607407,193.436890,...,0.2335,0.017962,13,1.727,0.061679,2.524,0.090143,-1.3356,-0.047700,28
4,4,2020-02-04 00:00:00,9292.841797,9331.265625,9112.811523,9180.962891,9180.962891,29893183716,189.861725,191.111496,...,-1.8571,-0.132650,14,1.235,0.044107,0.836,0.029857,0.6953,0.024832,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,727,2022-01-27 00:00:00,36841.878906,37148.324219,35629.281250,37138.234375,37138.234375,25041426629,2467.188477,2510.451660,...,1.1635,0.012378,94,7.581,0.076576,5.348,0.054020,5.7325,0.057904,99
728,728,2022-01-28 00:00:00,37128.445313,37952.878906,36211.109375,37784.332031,37784.332031,22238830523,2421.646973,2548.778564,...,0.1558,0.001513,103,4.335,0.046117,6.880,0.073191,-5.2646,-0.056006,94
729,729,2022-01-29 00:00:00,37780.714844,38576.261719,37406.472656,38138.179688,38138.179688,17194183075,2546.590576,2627.609375,...,1.7453,0.021816,80,5.498,0.063930,5.224,0.060744,0.7624,0.008865,86
730,730,2022-01-30 00:00:00,38151.917969,38266.339844,37437.710938,37917.601563,37917.601563,14643548444,2598.564941,2631.408447,...,-1.3114,-0.013114,100,6.444,0.067832,7.223,0.076032,-1.3598,-0.014314,95


In [8]:
for scoreset in scorelist2:
  print(scoreset)
  print(dfbase, "\n\n", dfbase.columns)
  dfAdd = pd.read_csv(scorelist2[scoreset])
  print(dfAdd)
  dfbase[scoreset + "PosScoresum"] = 0
  dfbase[scoreset + "PosScoreaverage"] = 0
  dfbase[scoreset + "NegScoresum"] = 0
  dfbase[scoreset + "NegScoreaverage"] = 0
  dfbase[scoreset + "ComScoresum"] = 0
  dfbase[scoreset + "ComScoreaverage"] = 0

  dfbase[scoreset + "Scorecount"] = 0

  for i in range(len(dfAdd)):
    ind = (trans(dfAdd.loc[i,"date"])-start).days
    if ind < 0 or ind > 731:
      pass
    else:

      dfbase.loc[ind, scoreset + "PosScoresum"] = dfbase.loc[ind, scoreset + "PosScoresum"] + dfAdd.loc[i,"pos"] #depends on if we only use compound or others too
      dfbase.loc[ind, scoreset + "NegScoresum"] = dfbase.loc[ind, scoreset + "NegScoresum"] + dfAdd.loc[i,"neg"] #depends on if we only use compound or others too
      dfbase.loc[ind, scoreset + "ComScoresum"] = dfbase.loc[ind, scoreset + "ComScoresum"] + dfAdd.loc[i,"compound"] #depends on if we only use compound or others too
      dfbase.loc[ind, scoreset + "Scorecount"] = dfbase.loc[ind, scoreset + "Scorecount"] + 1
  
  for i in range(len(dfbase)):
    if not dfbase.loc[i,scoreset + "Scorecount"] == 0:
      dfbase[scoreset + "PosScoreaverage"] = dfbase[scoreset + "PosScoresum"]/dfbase[scoreset + "Scorecount"]
      dfbase[scoreset + "NegScoreaverage"] = dfbase[scoreset + "NegScoresum"]/dfbase[scoreset + "Scorecount"]
      dfbase[scoreset + "ComScoreaverage"] = dfbase[scoreset + "ComScoresum"]/dfbase[scoreset + "Scorecount"]

reddit-bitcoin
     Unnamed: 0                 Date      BTC-Open      BTC-High  \
0             0  2020-01-31 00:00:00   9508.313477   9521.706055   
1             1  2020-02-01 00:00:00   9346.357422   9439.323242   
2             2  2020-02-02 00:00:00   9389.820313   9468.797852   
3             3  2020-02-03 00:00:00   9344.683594   9540.372070   
4             4  2020-02-04 00:00:00   9292.841797   9331.265625   
..          ...                  ...           ...           ...   
727         727  2022-01-27 00:00:00  36841.878906  37148.324219   
728         728  2022-01-28 00:00:00  37128.445313  37952.878906   
729         729  2022-01-29 00:00:00  37780.714844  38576.261719   
730         730  2022-01-30 00:00:00  38151.917969  38266.339844   
731         731  2022-01-31 00:00:00  37920.281250  38647.261719   

          BTC-Low     BTC-Close  BTC-Adj Close   BTC-Volume     ETH-Open  \
0     9230.776367   9350.529297    9350.529297  29432489719   184.736908   
1     9313.23925

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  del sys.path[0]


reddit-pandemic
     Unnamed: 0                 Date      BTC-Open      BTC-High  \
0             0  2020-01-31 00:00:00   9508.313477   9521.706055   
1             1  2020-02-01 00:00:00   9346.357422   9439.323242   
2             2  2020-02-02 00:00:00   9389.820313   9468.797852   
3             3  2020-02-03 00:00:00   9344.683594   9540.372070   
4             4  2020-02-04 00:00:00   9292.841797   9331.265625   
..          ...                  ...           ...           ...   
727         727  2022-01-27 00:00:00  36841.878906  37148.324219   
728         728  2022-01-28 00:00:00  37128.445313  37952.878906   
729         729  2022-01-29 00:00:00  37780.714844  38576.261719   
730         730  2022-01-30 00:00:00  38151.917969  38266.339844   
731         731  2022-01-31 00:00:00  37920.281250  38647.261719   

          BTC-Low     BTC-Close  BTC-Adj Close   BTC-Volume     ETH-Open  \
0     9230.776367   9350.529297    9350.529297  29432489719   184.736908   
1     9313.2392

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragme

reddit-politics
     Unnamed: 0                 Date      BTC-Open      BTC-High  \
0             0  2020-01-31 00:00:00   9508.313477   9521.706055   
1             1  2020-02-01 00:00:00   9346.357422   9439.323242   
2             2  2020-02-02 00:00:00   9389.820313   9468.797852   
3             3  2020-02-03 00:00:00   9344.683594   9540.372070   
4             4  2020-02-04 00:00:00   9292.841797   9331.265625   
..          ...                  ...           ...           ...   
727         727  2022-01-27 00:00:00  36841.878906  37148.324219   
728         728  2022-01-28 00:00:00  37128.445313  37952.878906   
729         729  2022-01-29 00:00:00  37780.714844  38576.261719   
730         730  2022-01-30 00:00:00  38151.917969  38266.339844   
731         731  2022-01-31 00:00:00  37920.281250  38647.261719   

          BTC-Low     BTC-Close  BTC-Adj Close   BTC-Volume     ETH-Open  \
0     9230.776367   9350.529297    9350.529297  29432489719   184.736908   
1     9313.2392

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragme

In [10]:
dfbase

,Unnamed: 0,Date,BTC-Open,BTC-High,BTC-Low,BTC-Close,BTC-Adj Close,BTC-Volume,ETH-Open,ETH-High,...,reddit-pandemicComScoresum,reddit-pandemicComScoreaverage,reddit-pandemicScorecount,reddit-politicsPosScoresum,reddit-politicsPosScoreaverage,reddit-politicsNegScoresum,reddit-politicsNegScoreaverage,reddit-politicsComScoresum,reddit-politicsComScoreaverage,reddit-politicsScorecount
0,0,2020-01-31 00:00:00,9508.313477,9521.706055,9230.776367,9350.529297,9350.529297,29432489719,184.736908,185.405838,...,3.2440,0.811000,4,105.292,0.127011,90.150,0.108745,80.5780,0.097199,829
1,1,2020-02-01 00:00:00,9346.357422,9439.323242,9313.239258,9392.875000,9392.875000,25922656496,180.113770,183.845551,...,0.0000,NaN,0,90.497,0.128001,84.876,0.120051,32.1929,0.045535,707
2,2,2020-02-02 00:00:00,9389.820313,9468.797852,9217.824219,9344.365234,9344.365234,30835736946,183.532501,193.080399,...,0.9538,0.953800,1,86.129,0.126474,79.325,0.116483,57.2687,0.084095,681
3,3,2020-02-03 00:00:00,9344.683594,9540.372070,9248.633789,9293.521484,9293.521484,30934096509,188.607407,193.436890,...,1.3616,0.680800,2,96.977,0.131583,82.521,0.111969,62.4516,0.084738,737
4,4,2020-02-04 00:00:00,9292.841797,9331.265625,9112.811523,9180.962891,9180.962891,29893183716,189.861725,191.111496,...,-1.9347,-0.644900,3,113.346,0.138565,87.845,0.107390,86.9004,0.106235,818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,727,2022-01-27 00:00:00,36841.878906,37148.324219,35629.281250,37138.234375,37138.234375,25041426629,2467.188477,2510.451660,...,2.2531,0.450620,5,61.515,0.115847,66.700,0.125612,-8.2675,-0.015570,531
728,728,2022-01-28 00:00:00,37128.445313,37952.878906,36211.109375,37784.332031,37784.332031,22238830523,2421.646973,2548.778564,...,-8.3716,-0.380527,22,65.160,0.118043,58.695,0.106332,37.5748,0.068070,552
729,729,2022-01-29 00:00:00,37780.714844,38576.261719,37406.472656,38138.179688,38138.179688,17194183075,2546.590576,2627.609375,...,0.8587,0.858700,1,61.324,0.118158,64.365,0.124017,-28.6265,-0.055157,519
730,730,2022-01-30 00:00:00,38151.917969,38266.339844,37437.710938,37917.601563,37917.601563,14643548444,2598.564941,2631.408447,...,0.0000,NaN,0,55.537,0.126508,58.756,0.133841,-29.8882,-0.068082,439


In [11]:
dfbase.to_csv("/content/drive/MyDrive/ISEF/Datasets/MergedPriceData.csv", index=False)

In [12]:
columns = list(dfbase.columns)[1:]
print(columns)

['Date', 'BTC-Open', 'BTC-High', 'BTC-Low', 'BTC-Close', 'BTC-Adj Close', 'BTC-Volume', 'ETH-Open', 'ETH-High', 'ETH-Low', 'ETH-Close', 'ETH-Adj Close', 'ETH-Volume', 'DOGE-Open', 'DOGE-High', 'DOGE-Low', 'DOGE-Close', 'DOGE-Adj Close', 'DOGE-Volume', 'AAPL-Open', 'AAPL-High', 'AAPL-Low', 'AAPL-Close', 'AAPL-Adj Close', 'AAPL-Volume', 'AMC-Open', 'AMC-High', 'AMC-Low', 'AMC-Close', 'AMC-Adj Close', 'AMC-Volume', 'DJI-Open', 'DJI-High', 'DJI-Low', 'DJI-Close', 'DJI-Adj Close', 'DJI-Volume', 'GME-Open', 'GME-High', 'GME-Low', 'GME-Close', 'GME-Adj Close', 'GME-Volume', 'NASDAQ-Open', 'NASDAQ-High', 'NASDAQ-Low', 'NASDAQ-Close', 'NASDAQ-Adj Close', 'NASDAQ-Volume', 'S&P-Open', 'S&P-High', 'S&P-Low', 'S&P-Close', 'S&P-Adj Close', 'S&P-Volume', 'TSLA-Open', 'TSLA-High', 'TSLA-Low', 'TSLA-Close', 'TSLA-Adj Close', 'TSLA-Volume', 'news-bitcoinPosScoresum', 'news-bitcoinPosScoreaverage', 'news-bitcoinNegScoresum', 'news-bitcoinNegScoreaverage', 'news-bitcoinComScoresum', 'news-bitcoinComScorea

In [13]:
dfout = dfbase[columns]

In [14]:
dfout

,Date,BTC-Open,BTC-High,BTC-Low,BTC-Close,BTC-Adj Close,BTC-Volume,ETH-Open,ETH-High,ETH-Low,...,reddit-pandemicComScoresum,reddit-pandemicComScoreaverage,reddit-pandemicScorecount,reddit-politicsPosScoresum,reddit-politicsPosScoreaverage,reddit-politicsNegScoresum,reddit-politicsNegScoreaverage,reddit-politicsComScoresum,reddit-politicsComScoreaverage,reddit-politicsScorecount
0,2020-01-31 00:00:00,9508.313477,9521.706055,9230.776367,9350.529297,9350.529297,29432489719,184.736908,185.405838,176.296814,...,3.2440,0.811000,4,105.292,0.127011,90.150,0.108745,80.5780,0.097199,829
1,2020-02-01 00:00:00,9346.357422,9439.323242,9313.239258,9392.875000,9392.875000,25922656496,180.113770,183.845551,179.745178,...,0.0000,NaN,0,90.497,0.128001,84.876,0.120051,32.1929,0.045535,707
2,2020-02-02 00:00:00,9389.820313,9468.797852,9217.824219,9344.365234,9344.365234,30835736946,183.532501,193.080399,180.173950,...,0.9538,0.953800,1,86.129,0.126474,79.325,0.116483,57.2687,0.084095,681
3,2020-02-03 00:00:00,9344.683594,9540.372070,9248.633789,9293.521484,9293.521484,30934096509,188.607407,193.436890,188.012695,...,1.3616,0.680800,2,96.977,0.131583,82.521,0.111969,62.4516,0.084738,737
4,2020-02-04 00:00:00,9292.841797,9331.265625,9112.811523,9180.962891,9180.962891,29893183716,189.861725,191.111496,185.403625,...,-1.9347,-0.644900,3,113.346,0.138565,87.845,0.107390,86.9004,0.106235,818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,2022-01-27 00:00:00,36841.878906,37148.324219,35629.281250,37138.234375,37138.234375,25041426629,2467.188477,2510.451660,2328.452148,...,2.2531,0.450620,5,61.515,0.115847,66.700,0.125612,-8.2675,-0.015570,531
728,2022-01-28 00:00:00,37128.445313,37952.878906,36211.109375,37784.332031,37784.332031,22238830523,2421.646973,2548.778564,2367.833740,...,-8.3716,-0.380527,22,65.160,0.118043,58.695,0.106332,37.5748,0.068070,552
729,2022-01-29 00:00:00,37780.714844,38576.261719,37406.472656,38138.179688,38138.179688,17194183075,2546.590576,2627.609375,2526.989746,...,0.8587,0.858700,1,61.324,0.118158,64.365,0.124017,-28.6265,-0.055157,519
730,2022-01-30 00:00:00,38151.917969,38266.339844,37437.710938,37917.601563,37917.601563,14643548444,2598.564941,2631.408447,2550.460938,...,0.0000,NaN,0,55.537,0.126508,58.756,0.133841,-29.8882,-0.068082,439


In [15]:
dfbase.to_csv("/content/drive/MyDrive/ISEF/Datasets/MergedPriceData.csv", index=False)